In [1]:
# Read csv file in './expors/FOLDER' into pandas dataframe
# Process dataframe
# Convert it into SQL insert and save to a file

In [2]:
# Config
## Where to save the joined csv
FOLDER = './exports/2024-08'
## Tx from these wallet will be proccessed as shared tx (ammount = amount * SHARE_RATIO)
SHARE_WALLETS = ['Shared Sparkasse', 'Revolut Joint']
SHARE_RATIO = 0.533333
## Transaction with this tag will be considered shared transaction too
SHARED_TAG = 'Shared'
OUTPUT_FILE = './2024-08.sql'

In [3]:
# util function to return insert sql script as string
import re

def get_insert_query_from_df(df, dest_table):

    insert = """
    INSERT INTO {dest_table} (
        """.format(dest_table=dest_table)

    columns_string = str(list(df.columns))[1:-1]
    columns_string = re.sub(r' ', '\n        ', columns_string)
    columns_string = re.sub(r'\'', '', columns_string)

    values_string = ''

    for row in df.itertuples(index=False,name=None):
        values_string += re.sub(r'nan', 'null', str(row))
        values_string += ',\n'

    return insert + columns_string + ')\n     VALUES\n' + values_string[:-2] + ';'

In [4]:
# Read all csv file in folder
import pandas as pd
import glob
import os

# setting the path for joining multiple files
files = os.path.join(FOLDER, '*.csv')

# list of merged files returned
files = glob.glob(files)

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)

In [5]:
# Copy amount to real_amount
df['real_amount'] = df['Amount']

In [6]:
# Scale down amount from real amount to account for shared expense
# If is Shared wallet
df.loc[df['Wallet'].isin(SHARE_WALLETS), 'Amount'] = round(df['real_amount']*SHARE_RATIO, 2)
# If has shared tag
df.loc[df['Labels'] == SHARED_TAG, 'Amount'] = round(df['real_amount']*SHARE_RATIO, 2)


# Set as Incoming Transfer If is extra income (from Vic) in Sparkasse
df.loc[(df['Wallet'].isin(SHARE_WALLETS)) & (df['Category name'] == 'Extra Income'), 'Type'] = 'Incoming Transfer'


In [7]:
# Clean up column
del df['Author']
del df['Currency']
df = df.rename(columns={
    'Date': 'date',
    'Wallet': 'wallet',
    'Type': 'type',
    'Category name': 'category',
    'Amount': 'amount',
    'Note': 'note',
    'Labels': 'labels'
})

In [8]:
# Move columns
df=df[["date","wallet","type","category", "amount", "real_amount", "note", "labels"]]

In [9]:
# Generate Insert script
file1 = open(OUTPUT_FILE, 'w')
file1.write(get_insert_query_from_df(df, 'transactions'))
file1.close()